In [ ]:
# ─────────────────────────── imports ───────────────────────────
import os, logging
import numpy as np
from scipy.io import wavfile
from scipy.signal import hilbert, stft
import ordpy
import soundfile as sf
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import antropy as ent
import librosa


In [ ]:
# ─────────────────────────── logger ───────────────────────────
logger = logging.getLogger(__name__)

In [ ]:

# ──────────────────────── 1. Audio Loader ──────────────────────
def _load_mono_audio(file_path):
    """
    Считываем WAV (scipy) или любой другой формат (soundfile/librosa backend)
    и нормализуем до float32 [-1, 1].
    """
    ext = os.path.splitext(file_path)[1].lower()
    if ext == ".wav":
        sr, data = wavfile.read(file_path)
        if data.ndim == 2:
            data = data.mean(axis=1)
        data = data.astype(np.float32)
        if data.max() > 1.0:          # int16 → float
            data /= np.abs(data).max() + 1e-9
        logger.info(f"WAV '{file_path}' loaded, sr={sr}")
    else:  # mp3, flac, …
        data, sr = sf.read(file_path, always_2d=False)
        if data.ndim == 2:
            data = data.mean(axis=1)
        if data.dtype != np.float32:
            data = data.astype(np.float32)
        logger.info(f"Audio '{file_path}' loaded via soundfile, sr={sr}")
    return data, sr


In [ ]:
# ───────────────────────── helpers ────────────────────────────
def _frame_signal(x, frame_len, hop_len):
    """np.lib.stride_tricks.sliding_window_view wrapper → shape (n_frames, frame_len)"""
    if len(x) < frame_len:
        return np.empty((0, frame_len))
    frames = np.lib.stride_tricks.sliding_window_view(x, frame_len)[::hop_len]
    return frames


def _ordpy_pe_ce(ts, dim, tau):
    """Безопасный вызов ordpy.complexity_entropy с нормализацией."""
    if len(ts) <= dim:
        return np.nan, np.nan
    H, C = ordpy.complexity_entropy(ts, dx=dim, taux=tau)
    return H, C


In [ ]:
# ═══════════════════ 2. Амплитудный хаос ═══════════════════════
def ordpy_amplitude(file_path, dim_size=7, hop_size=1,
                    win_len=2048, hop_len=1024):
    """Hilbert envelope → окно-среднее → PE-CE."""
    x, sr = _load_mono_audio(file_path)

    env = np.abs(hilbert(x))
    frames = _frame_signal(env, win_len, hop_len)
    env_mean = frames.mean(axis=1)          # 1-D ряд

    return _ordpy_pe_ce(env_mean, dim_size, hop_size)


In [ ]:
# ═══════════════════ 3. Тембровый флюкс ════════════════════════
def ordpy_flux(file_path, dim_size=6, hop_size=2,
               n_fft=2048, hop_len=512, smooth=7):
    """
    STFT через scipy.signal.stft → spectral flux →
    ordpy PE-CE.
    """
    x, sr = _load_mono_audio(file_path)

    f, t, Z = stft(x, fs=sr, nperseg=n_fft, noverlap=n_fft - hop_len,
                   window="hann", padded=False, boundary=None)
    mag = np.abs(Z)                         # shape (n_bins, n_frames)
    if mag.shape[1] < 2:
        return np.nan, np.nan
      # ── FLUX ──────────────────────────────────────────────────────────
    # 1) лог-магнитуда смягчает пики
    mag_log = np.log1p(np.abs(Z))
    diff    = np.diff(mag_log, axis=1)
    flux    = np.sqrt((diff**2).sum(axis=0))   # 1-D

    # 2) сглаживание (rolling mean)
    if smooth > 1:
        kernel = np.ones(smooth, dtype=float) / smooth
        flux = np.convolve(flux, kernel, mode="valid")
    diff = np.diff(mag, axis=1)

    return _ordpy_pe_ce(flux, dim_size, hop_size)

In [ ]:
# ═══════════════════ 4. Гармонический хаос ═════════════════════
def ordpy_harmony(file_path, dim_size=6, hop_size=1,
                  n_fft=4096, hop_len=1024):
    """
    Быстрая самодельная chroma без librosa.feature:
    STFT → каждый бин → pitch-class (mod 12) →
    энергия семикварта-круга → угловой ряд → PE-CE.
    """
    x, sr = _load_mono_audio(file_path)

    f, t, Z = stft(x, fs=sr, nperseg=n_fft, noverlap=n_fft - hop_len,
                   window="hann", padded=False, boundary=None)
    mag = np.abs(Z) ** 2

    # Pitch-class для каждого част. бина (>50 Гц)
    eps = 1e-12
    valid = f > 50
    f = f[valid]
    mag = mag[valid]

    # MIDI номер + мод 12
    midi = 69 + 12 * np.log2(f / 440.0 + eps)
    pc = np.mod(np.round(midi).astype(int), 12)   # 0–11

    # ▸ Накопление энергии в 12 хромах
    chroma = np.zeros((12, mag.shape[1]), dtype=np.float32)
    for k in range(12):
        chroma[k] = mag[pc == k].sum(axis=0) if np.any(pc == k) else 0.0

    # ▸ Вектор на круге кварто-квинт
    angles = np.arange(12) * 2 * np.pi / 12
    vector = (chroma * np.exp(1j * angles[:, None])).sum(axis=0)
    harmonic_angle = np.angle(vector)             # 1-D

    return _ordpy_pe_ce(harmonic_angle, dim_size, hop_size)
